In [22]:
import numpy as np
import pandas as pd

In [23]:
file_path = '/home/jeremy/Documents/Greater MSP Challenge'
file_name = file_path + '/Historical Data - Prior Dashboard - Color Guides/2015-2019 Dashboard Trends_all.xlsx'

In [24]:
dfs = pd.read_excel(file_name, sheet_name=None, header=None)

In [25]:
dfs

{'Key Indicators':                                                    0                    1   \
 0   About: Note all indicators in this dashboard a...                  NaN   
 1                                                 NaN                  NaN   
 2                                      Key Indicators                  NaN   
 3                                  Dashboard Category              Economy   
 4                                     Indicator Title    Yearly Job Growth   
 5                  Dashboard Year\n(Data Source Year)   2015\n(12-13 data)   
 6        Atlanta-Sandy Springs-Roswell, GA Metro Area                0.027   
 7                    Austin-Round Rock, TX Metro Area                0.041   
 8           Boston-Cambridge-Newton, MA-NH Metro Area                0.017   
 9        Charlotte-Concord-Gastonia, NC-SC Metro Area                0.209   
 10      Chicago-Naperville-Elgin, IL-IN-WI Metro Area                0.015   
 11         Dallas-Fort Worth-Arli

### Each sheet of the Excel workbook is represented by a key/value pair in dfs

In [26]:
dfs.keys()

dict_keys(['Key Indicators', 'Economy', 'BV', 'Talent', 'Education', 'Infrastructure', 'Environment', 'Livability', 'Vital Stats'])

## The Key Indicators sheet has the stats that appear in the pamphlet.
It also has an extra row, "Dashboard Category", that does not exist in the other sheets. This extra row tells us which sheet the stats are coming from.

The stats on the Key Indicators sheet are doubled up on the other sheets, so we don't need to capture them. The important info to capture is which stats are listed, so when we get to the individual sheets, we know which stats are key indicators.

The remaining sheets represent the data behind each set of stats on the Key Indicators sheet, and additional indicators. They seem to have the same structure, so looping through them will be easier.

In [27]:
dfs['Key Indicators'].loc[[3,4]]

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
3,Dashboard Category,Economy,NaN,NaN,NaN,NaN,Economy,NaN,NaN,NaN,...,Livability,NaN,NaN,NaN,NaN,Livability,NaN,NaN,NaN,NaN
4,Indicator Title,Yearly Job Growth,NaN,NaN,NaN,NaN,Employment Gap (White-Of Color),NaN,NaN,NaN,...,Percent of Cost-Burdened Households,NaN,NaN,NaN,NaN,Annual Change in Median Apartment Rent (Overall),NaN,NaN,NaN,NaN


In [28]:
kpis = dfs['Key Indicators'].loc[[3,4]].dropna(axis='columns')
kpis

,0,1,6,11,16,21,26,31,36,41,46,51,56,61,66,71
3,Dashboard Category,Economy,Economy,Business Vitality,Business Vitality,Talent,Talent,Talent,Education,Education,Infrastructure,Infrastructure,Environment,Environment,Livability,Livability
4,Indicator Title,Yearly Job Growth,Employment Gap (White-Of Color),New Establishments (State Level),Establishments Surviving 5 Years (State Level),Net Migration Of 25-34 Year Olds,Females Aged 16-64 Years Working,Population 25+ with Bachelor's Degree or Higher,3-Year Graduation Rate at 2-Year Institutions ...,6-Year Graduation Rate at 4-Year Institutions ...,"Population Living Within 30 Minutes of 100,000...",Population With Commutes Less Than 30 Minutes,Energy Related Carbon Dioxide Emissions Per Ca...,Electricity Produced From Non-Carbon Sources,Percent of Cost-Burdened Households,Annual Change in Median Apartment Rent (Overall)


### Capture the Dashboard Category and Indicator Title of each indicator.
Store them in a dictionary we can reference later.

In [29]:
kpi_dict = {}
for col in kpis.columns:
    if col == 0:
        continue
    category = kpis[col].iloc[0]
    indicator = kpis[col].iloc[1]
    if category in kpi_dict:
        kpi_dict[category].append(indicator)
    else:
        kpi_dict[category] = [indicator]

kpi_dict

{'Economy': ['Yearly Job Growth', 'Employment Gap (White-Of Color)'],
 'Business Vitality': ['New Establishments (State Level)',
  'Establishments Surviving 5 Years (State Level)'],
 'Talent': ['Net Migration Of 25-34 Year Olds',
  'Females Aged 16-64 Years Working',
  "Population 25+ with Bachelor's Degree or Higher"],
 'Education': ['3-Year Graduation Rate at 2-Year Institutions (State-Level)',
  '6-Year Graduation Rate at 4-Year Institutions (State Level)'],
 'Infrastructure': ['Population Living Within 30 Minutes of 100,000 Jobs By Transit or Walking',
  'Population With Commutes Less Than 30 Minutes'],
 'Environment': ['Energy Related Carbon Dioxide Emissions Per Capita (State Level)',
  'Electricity Produced From Non-Carbon Sources'],
 'Livability': ['Percent of Cost-Burdened Households',
  'Annual Change in Median Apartment Rent (Overall)']}

### The key indicators on the first sheet of the Excel document don't always line up with the key indicators in the pamphlet.

## For the remaining sheets:
### Value in first row/first column is the sheet name/key indicator name (ex: "Economy")
We already have this information from `dfs.keys()`.

### First two rows are headers:
1. Indicator (ex: Gross Regional Product Growth). This is only populated when it changes, NaN otherwise.
2. "ESTIMATES". This is the year for each Indicator, mostly represented as '4-digit year\n(year of data)'.

### Cities show up twice on each sheet--first time with the observation, second time with the rank.
There's a "RANK" row separating the two. This row indicates whether the ranking is "highest to lowest", or "lowest to highest". Capture this.

In [30]:
dfs['Economy']

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,Economy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Gross Regional Product Growth,NaN,NaN,NaN,NaN,Yearly Job Growth,NaN,NaN,NaN,...,Wage Gap (White-Of Color),NaN,NaN,NaN,NaN,Employment Gap (White-Of Color),NaN,NaN,NaN,NaN
2,ESTIMATES,2015\n(12-13 data),2016\n(13-14 data),2017\n(14-15 data),2018\n(15-16 data),2019\n(16-17 data),2015\n(12-13 data),2016\n(13-14 data),2017\n(14-15 data),2018\n(15-16 data),...,2015\n(2013 data),2016\n(2014 data),2017\n(2015 data),2018\n(2016 data),2019 \n(2017 data),2015\n(2013 data),2016\n(2014 data),2017\n(2015 data),2018\n(2016 data),2019 \n(2017 data)
3,"Atlanta-Sandy Springs-Roswell, GA Metro Area",0.035,0.04,0.038,0.044,0.029,0.0267236,0.0329032,0.0324358,0.0320127,...,0.655,0.681818,0.666667,0.659574,0.666667,0.05511,0.0672503,0.0530567,0.0368609,0.0396132
4,"Austin-Round Rock, TX Metro Area",0.06,0.054,0.083,0.055,0.069,0.0408405,0.0427223,0.0466901,0.033,...,0.65,0.62212,0.650759,0.65,0.65625,0.047021,0.063918,0.0511267,0.0503152,0.0341938
5,"Boston-Cambridge-Newton, MA-NH Metro Area",0.005,0.022,0.044,0.011,0.028,0.0168563,0.0202838,0.0228392,0.0205694,...,0.666667,0.6,0.62,0.673077,0.673077,0.0718597,0.0902325,0.0791438,0.076,0.0647034
6,"Charlotte-Concord-Gastonia, NC-SC Metro Area",0.01,0.025,0.052,0.037,0.035,0.209392,0.0363886,0.04,0.0366681,...,0.675676,0.625,0.6875,0.702439,0.714286,-0.001,-0.013,0.0408677,0.0492271,0.0236565
7,"Chicago-Naperville-Elgin, IL-IN-WI Metro Area",-0.003,0.017,0.025,0.014,0.015,0.0145957,0.0169556,0.0189012,0.0135039,...,0.613636,0.637363,0.625,0.6,0.6,0.123306,0.118448,0.105498,0.0992639,0.0854895
8,"Dallas-Fort Worth-Arlington, TX Metro Area",0.054,0.046,0.055,0.015,0.039,0.0346941,0.034083,0.0361918,0.0271718,...,0.568182,0.583333,0.583333,0.6,0.6,0.0561441,0.06,0.0443108,0.0429154,0.0468995
9,"Denver-Aurora-Lakewood, CO Metro Area",0.013,0.06,0.046,0.012,0.036,0.0370241,0.0389417,0.0383021,0.0278119,...,0.679612,0.655022,0.638298,0.6,0.653061,0.0773576,0.0861799,0.0834035,0.076,0.0539857


## Functions

### The year field often has an appended text string, indicating the actual date ranges involved.
Strip that extra text, if necessary.

In [31]:
def cleanup_year(string):
    try:
# split will fail when the value is a lone year (ex: 2015 vs. "2015\n(using 13-14 data)")
# because the lone year is interpreted as an int. Rather than convert that to a string,
# just return it.
        return string.split()[0]
    except:
        return string
    else:
        return string

### Set the Key Indicator to 1 for key indicators.

In [32]:
def set_key_indc(category,indicator,**kwargs):
    my_kpi_dict = kwargs.get('kpi_dict',kpi_dict)
    if category in my_kpi_dict:
        if indicator in kpi_dict[category]:
            return 1
    return 0  

### Alteryx logic for deciding Data Type:
```
if abs([Measure Value]) <=1 then 'Percent'
elseif Contains([Indicator Title], 'Cost') then 'Dollar'
elseif Contains([Indicator Title], 'Wage') then 'Dollar'
elseif Contains([Indicator Title], 'Income') then 'Dollar'
elseif Contains([Indicator Title], 'Price') then 'Dollar'
elseif [Dashboard Category] = 'Business Vitality' 
	AND !(Contains([Indicator Title], 'Patent')
	or Contains([Indicator Title], 'Establishment'))
	then 'Dollar'
else 'Numeric'
endif 
```

In [33]:
# This is where we implement Dave's logic from Alteryx.
def calculate_data_type(series):
    if series['Value'] <=1:
        return 'Percent'
    elif any(x in series['Indicator'].lower() for x in ['cost','wage','income','price']):
        return 'Dollar'
    elif (series['Category'] == 'Business Vitality') \
         and not any(x in series['Indicator'].lower() for x in ['patent','establishment']):
        return 'Dollar'
    else:
        return "Numeric"

### Figure out the type of value for each category/indicator.
* Build a small dataframe of unique combos of Category + Indicator, and the max Value for each combo. (I checked--max() ignores NaNs.)
* Convert that dataframe into a dictionary containing Indicator/Data Type pairs.

In [34]:
def build_data_type_df(df):
    this_category = df['Category'].unique()[0]
    df_cat = df.loc[df['Category'] == this_category]
# There is something funky about these two sheets--the .groupby() doesn't behave the same way.
# I found that if I drop category, it works better.
    if this_category in ['Environment','Livability']:
        max_df = pd.DataFrame()
        for this_indicator in df_cat['Indicator'].unique():
            indicator_data_df = df_cat.loc[df['Indicator'] == this_indicator,['Indicator','Value']]
            append_df = indicator_data_df.loc[:,['Indicator','Value']].groupby(['Indicator']).max().reset_index()
            max_df = max_df.append(append_df,ignore_index=True)
# Add back the dropped Category column, so calculate_data_type() works.
        max_df['Category'] = [this_category] * max_df.shape[0]
    else:
        max_df = df.loc[:,['Category','Indicator','Value']].groupby(['Category','Indicator']).max().reset_index()
    max_df['Value'] = max_df['Value'].apply(lambda x: abs(x))
    max_df['Data_Type'] = max_df.apply(lambda row: calculate_data_type(row),axis='columns')

# Prep the dataframe so it can be easily transformed into a dictionary
    max_df.drop(columns=['Category','Value'],inplace=True)

    data_type_df = max_df.set_index(['Indicator']).to_dict('index')
# I couldn't find a parameter for .to_dict() that gave me exactly what I wanted--
# namely, a dict where key is Indicator, value is Data Type.
# So build a new dict that has this format, and extract what we need from data_type_df.
    return_dict = {}
    for this_key in data_type_df:
        return_dict[this_key] = data_type_df[this_key]['Data_Type']
    return return_dict

## Restructure the data for each sheet.
### Each row of the data should have the following columns:
1. Category
2. Indicator
3. Metro area
4. Year description (as taken from sheet)
5. Year (integer value only)
6. Value
7. Rank
8. Rank Order
9. Key Indicator
10. Data Type (type of Value; one of: 'Percent','Dollar','Numeric')

In [35]:
def reshape_indicator_sheet(df):
    out_df_columns = ['Category','Indicator','Metro','Year_Desc','Year',
                      'Value','Rank','Rank_Order','Key_Indicator']
    out_df = pd.DataFrame(columns=out_df_columns)
# Cast the type of the numeric columns, so we can run some basic calculations and set 'Data Type'
#    out_df = out_df.astype({'Year': 'Int64', 'Value': 'float', 'Rank': 'Int64', 'Key_Indicator': 'Int64'})

# Capture the index of the RANK row.
# This will help us split the sheet into its values vs. rank halves.
    rank_index = np.where(df[0] == 'RANK')[0][0]
    indicators_sheet = df.iloc[0:rank_index].copy()
    rank_sheet = df.iloc[rank_index:].copy()

# Loop through each city, snag its values and rank info, drop them into the correct columns
    for metro in indicators_sheet[0][3:]:
        city_df = pd.DataFrame(columns=out_df_columns)
        city_category = indicators_sheet.loc[[0]].values[0][0].strip()

# This will fill 'Category' with nulls, but it gives the dataframe the correct length.
# Then, fill it with the actual category name.
        city_df['Category'] = indicators_sheet.loc[[0]].values[0][1:]
        city_df['Category'].fillna(value=city_category,inplace=True)

# Indicator has some values populated, so we can use 'pad' to copy them forward to null rows.
        city_df['Indicator'] = indicators_sheet.loc[[1]].values[0][1:]
        city_df['Indicator'].fillna(method='pad',inplace=True)
        city_df['Indicator'] = city_df['Indicator'].apply(lambda x: x.strip())
        
# If we find Indicator in the KPI dictionary under this Category, this is a Key Indicator
        city_df['Key_Indicator'] = city_df.apply(lambda row: set_key_indc(row['Category'],row['Indicator']),
                                                 axis='columns')

        city_df['Year_Desc'] = indicators_sheet.iloc[2].values[1:]
        city_df['Year'] = city_df['Year_Desc'].apply(lambda x: cleanup_year(x))

        indc_metro_value = np.where(indicators_sheet[0] == metro)[0][0]
        city_df['Metro'] = [metro] * city_df.shape[0]
        city_values = indicators_sheet.iloc[indc_metro_value].values[1:]
        city_df['Value'] = city_values

# Find the index for our current Metro in the rank half of the sheet, and get Rank-related values.
        rank_metro_value = np.where(rank_sheet[0] == metro)[0][0]
        city_df['Rank'] = rank_sheet.iloc[rank_metro_value].values[1:]
        city_df['Rank_Order'] = rank_sheet.iloc[[0]].values[0][1:]
        city_df['Rank_Order'].fillna(method='pad',inplace=True)

# We're done with this city. Append its stats to the big spreadsheet.
        out_df = out_df.append(city_df,ignore_index=True)

    data_type_dict = build_data_type_df(out_df)
    out_df.loc[:,'Data_Type'] = out_df['Indicator'].apply(lambda x: data_type_dict[x])
    return out_df

## Concatenate the restructured output from all of the sheets (minus Key Indicator)

In [36]:
output_df = pd.DataFrame()
for sheet_key in dfs.keys():
    if sheet_key == 'Key Indicators':
        continue
    output_df = output_df.append(reshape_indicator_sheet(dfs[sheet_key]),ignore_index=True)

In [37]:
output_df['Data_Type'].value_counts()

Percent    2364
Numeric     912
Dollar      420
Name: Data_Type, dtype: int64

In [38]:
output_df.describe(include='all')

,Category,Indicator,Metro,Year_Desc,Year,Value,Rank,Rank_Order,Key_Indicator,Data_Type
count,3696,3696,3696,3624,3624,2954,2898,3276,3696,3696
unique,8,60,12,69,11,2123,12,3,2,3
top,Vital Stats,Population With Commutes Less Than 30 Minutes,"Denver-Aurora-Lakewood, CO Metro Area",2016\n(2014 data),2018,0,1,(highest to lowest),0,Percent
freq,804,132,308,240,672,22,261,2124,2724,2364


### Some things to note about the output from .describe() above

* "count" doesn't include null values, so the columns will have differing counts (the counts for Value and Rank in particular are a lot lower than the others). Overall row count makes sense...? I didn't drop rows with null Values, but that's maybe one extra line of code, if that's what the Alteryx solution does.

* All values are all being treated as strings. In Python, I can't convert Nulls to an int or float, so Year/Value/Rank can't be represented as numbers. If any of these columns were numbers, the stats would include numeric summary statistics, like min/max/mean.

* Unique counts helped me determine whether the conversion I performed above was done correctly. 8 Categories, 12 Metros, 11 Years, 12 Ranks, etc. Looks good...?

## Write a CSV file.

In [39]:
output_df.to_csv(file_path + '/greater_msp_data.csv',index=False)